# Returning Multiple Values in Singe Operate Call

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.*;

In [5]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.\n");


Initialized the client and connected to the cluster.



In [18]:
String ns = "test";
String setName = "testset";
int profileId = 2;
String name = "Aerospike";
int zip = 94506;
    
Key user = new Key(ns, setName, profileId); 
    
Bin bProfileId = new Bin("ProfileId", profileId);
Bin bName = new Bin("Name", name);
Bin bZip = new Bin("Zip", zip);
    
WritePolicy wPolicy = new WritePolicy();
wPolicy.durableDelete = false;
//wPolicy.expiration = -1;

//Create a record

client.put(wPolicy, user, bProfileId, bName, bZip);
      

Record rec = client.operate(wPolicy, user, 
                ExpOperation.read("retDeviceSize", Exp.build(Exp.deviceSize()), ExpReadFlags.DEFAULT),
                ExpOperation.read("retName", Exp.build(Exp.stringBin("Name")), ExpReadFlags.DEFAULT),            
                ExpOperation.read("retZip", Exp.build(Exp.intBin("Zip")), ExpReadFlags.DEFAULT),  
                //Operation.get() //NOT ALLOWED - but you can return individual bins 
                Operation.get("Name"),
                Operation.get("Zip")
             ) ; 

System.out.format("Record Device size: %d\n", rec.getValue("retDeviceSize"));
System.out.format("Record Name Bin: %s\n", rec.getValue("retName"));
System.out.format("Record Zip Bin: %d\n", rec.getValue("retZip"));
System.out.println("Name: "+rec.getValue("Name"));
System.out.println("Zip: "+rec.getValue("Zip"));

System.out.println("Check record data on server.\n");    
System.out.println(client.get(null, user));
    

Record Device size: 96
Record Name Bin: Aerospike
Record Zip Bin: 94506
Name: Aerospike
Zip: 94506
Check record data on server.

(gen:13),(exp:0),(bins:(ProfileId:2),(Name:Aerospike),(Zip:94506))


# Cleanup

In [7]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"